In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
!pip install numpy --break-system-packages -q
!pip install chromadb --break-system-packages -q
!pip install sentence-transformers --break-system-packages -q
!pip install langchain --break-system-packages -q
!pip install openai --break-system-packages -q
!pip install python-dotenv --break-system-packages -q
!pip install tiktoken  --break-system-packages -q
!pip install langchain-community --break-system-packages -q
!pip install datasets --break-system-packages -q
!pip install optimum --break-system-packages -q
!pip install bitsandbytes --break-system-packages -q
!pip install bert_score --break-system-packages -q
!pip install transformers --break-system-packages -q
!pip install trl --break-system-packages -q
!pip install PyPDF2 --break-system-packages -q

# Step 1 - Importing The Libraries

In [ ]:
# Basic Data Processing
import pandas as pd
import json

# PDF Processing
from PyPDF2 import PdfReader
import re

# Vector Database
import chromadb
from langchain.vectorstores import Chroma

# LangChain Components
from sentence_transformers import SentenceTransformer
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain.schema import Document

# Step 1 - PDF Processing and JSON Extraction

In [4]:
def process_pdf_to_json(pdf_path, output_json_path):
    """
    Process PDF file and extract content into JSON format
    """
    # Read PDF
    reader = PdfReader(pdf_path)
    
    # Initialize list to store all documents
    documents = []
    
    # Process each page
    for page_num, page in enumerate(reader.pages):
        text = page.extract_text()
        
        # Split text into sections (you might need to adjust this based on your PDF structure)
        sections = re.split(r'\n\s*\n', text)
        
        for section in sections:
            if len(section.strip()) > 50:  # Minimum content length
                # Create document structure
                doc = {
                    "Title": f"Page {page_num + 1} - Section",  # You might want to extract actual titles
                    "Context": section.strip(),
                    "Page": page_num + 1
                }
                documents.append(doc)
    
    # Save to JSON
    with open(output_json_path, 'w', encoding='utf-8') as f:
        json.dump(documents, f, ensure_ascii=False, indent=2)
    
    print(f"Processed {len(reader.pages)} pages into {len(documents)} documents")
    return documents

In [5]:
# Process the PDF and save to JSON
pdf_path = "inputs/data/raw/data.pdf"
json_output_path = "outputs/data/processed/data.json"

documents = process_pdf_to_json(pdf_path, json_output_path)

Processed 55 pages into 112 documents


# Step 2 - Loading and Restructuring JSON Data

In [6]:
def load_the_data(file_path):
    """Load and process the JSON file"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            
        print(f"Successfully loaded {len(data)} documents")
        return data
        
    except FileNotFoundError:
        print(f"File {file_path} not found!")
        return []
    except json.JSONDecodeError as e:
        print(f"JSON decode error: {e}")
        return []
    except Exception as e:
        print(f"Unexpected error: {e}")
        return []

def analyze_the_data(data):
    """Analyze the loaded data"""
    if not data:
        return
    
    # Create DataFrame
    df = pd.DataFrame(data)
    
    print("\nData Analysis:")
    print(f"Total documents: {len(df)}")
    print(f"Columns: {df.columns.tolist()}")
    
    # Calculate text lengths
    df['title_length'] = df['Title'].apply(len)
    df['context_length'] = df['Context'].apply(len)
    
    print(f"\nText Length Statistics:")
    print(f"Average title length: {df['title_length'].mean():.1f} characters")
    print(f"Average context length: {df['context_length'].mean():.1f} characters")
    print(f"Total context characters: {df['context_length'].sum():,}")
    
    return df

def display_documents(data, max_display=5):
    """Display the documents in a readable format"""
    print(f"\nDocuments (showing first {max_display}):")
    print("-" * 50)
    
    for i, doc in enumerate(data[:max_display]):
        print(f"\n{i+1}. {doc['Title']}")
        print(f"   {doc['Context'][:100]}...")
        print(f"   Length: {len(doc['Context'])} characters")
        print("-" * 30)

In [7]:
file_path = "outputs/data/processed/data.json"
    
# Load the data
whole_data = load_the_data(file_path)

Successfully loaded 112 documents


In [8]:
if whole_data:    
    # Display documents
    display_documents(whole_data)

    # Show Some Analysis
    analyze_the_data(whole_data)


Documents (showing first 5):
--------------------------------------------------

1. Page 1 - Section
   الوثيقة القانونية والتشريعات الصادرة لصاحل املرأة الصرية...
   Length: 56 characters
------------------------------

2. Page 2 - Section
   2 
  النظام القانوينجلهموورية صصر العريية   
السياق العام للبلد 
 العاصمة: القاهرة   
عدد السكان1 : ...
   Length: 1898 characters
------------------------------

3. Page 2 - Section
   1 - الجهاز المركزي للتعبئة العامة واالحصاء؛ الكتاب  االحصائي السنوي، سبتمبر 2015. 
2 - الجهاز المركز...
   Length: 205 characters
------------------------------

4. Page 3 - Section
   3 
  الناتج المحلىاإلجمالي4  المصري عن عام 2013/2014  = بسعر السوق 1643,4 .مليار جنية مصري 5 
 معدل ...
   Length: 180 characters
------------------------------

5. Page 3 - Section
   4  الناتج المحلى االجماليبسعر السوق: ما انتجه من سلع وخدمات باألسعار الجارية مستبعد فئة المستلزمات ا...
   Length: 428 characters
------------------------------

Data Analysis:
Total documents: 112
C

# Step 3 - Document Creation

In [9]:
def Create_documents(whole_data):
    """
    Create documents suitable for embedding models
    Returns: List of Document objects with combined text for embedding
    """
    documents = []
    
    for item in whole_data:
        if isinstance(item, list):
            item = item[0]

        title = item.get("Title", "")
        context = item.get("Context", "")

        if title and context:
            # Combine title and context for better embedding quality
            combined_text = f"{title}\n\n{context}"
            
            documents.append(Document(
                page_content=combined_text,  # This is what gets embedded
                metadata={
                    "title": title,
                    "context": context,
                    "text_length": len(combined_text)
                }
            ))
    
    return documents

In [10]:
# Usage:
documents = Create_documents(whole_data)

In [11]:
print(f"Documents lenght : {len(documents)}")

for i, doc in enumerate(documents[:2]):
    print(f"\nDocument {i+1}:")
    print(f"Title : {doc.page_content[:100]}...")
    print(f"Meta Data : {doc.metadata}")
    print("-" * 50)

Documents lenght : 112

Document 1:
Title : Page 1 - Section

الوثيقة القانونية والتشريعات الصادرة لصاحل املرأة الصرية...
Meta Data : {'title': 'Page 1 - Section', 'context': 'الوثيقة القانونية والتشريعات الصادرة لصاحل املرأة الصرية', 'text_length': 74}
--------------------------------------------------

Document 2:
Title : Page 2 - Section

2 
  النظام القانوينجلهموورية صصر العريية   
السياق العام للبلد 
 العاصمة: القاهرة...
Meta Data : {'title': 'Page 2 - Section', 'context': '2 \n  النظام القانوينجلهموورية صصر العريية   \nالسياق العام للبلد \n العاصمة: القاهرة   \nعدد السكان1 : 96,521,618 \nاللغة الرسمية: العربية \nالمساحة الجغرافية: تقع جمهورية مصر العربية فى الشمال الشرقي لقار ة \nإفريقيا، وتطل على كل من الساحل الجنوبي الشرقي للبحر المتوسط \nوالساحل الشمالي الغربي للبحر االحمر بمساحة اجمالية تبلغ مليون كم 2 \n تقريباً، مصر دولة إفريقية غير أن جزءا من أراضيها، وهى شبه جزيرة\nسيناء" يقع في قارة أسيا. \nلمصر حدود مشتركة مع ليبيا من الغرب، مع السودان من الجنوب، ومع \nاسرائيل وفلسطين (

In [29]:
# For embeddings, you'll use:
texts = [doc.page_content for doc in documents] 

print(f"Created {len(documents)} documents for embedding")
print(f"Sample text for embedding: {texts[1][:]}...")

Created 112 documents for embedding
Sample text for embedding: Page 2 - Section

2 
  النظام القانوينجلهموورية صصر العريية   
السياق العام للبلد 
 العاصمة: القاهرة   
عدد السكان1 : 96,521,618 
اللغة الرسمية: العربية 
المساحة الجغرافية: تقع جمهورية مصر العربية فى الشمال الشرقي لقار ة 
إفريقيا، وتطل على كل من الساحل الجنوبي الشرقي للبحر المتوسط 
والساحل الشمالي الغربي للبحر االحمر بمساحة اجمالية تبلغ مليون كم 2 
 تقريباً، مصر دولة إفريقية غير أن جزءا من أراضيها، وهى شبه جزيرة
سيناء" يقع في قارة أسيا. 
لمصر حدود مشتركة مع ليبيا من الغرب، مع السودان من الجنوب، ومع 
اسرائيل وفلسطين (قطاع غزة) من الشمال الشرقي، ويمر الممر المائي (قناة 
السويس) الذى يربط البحر المتوسط والبحر االحمر عبر االراضي المصرية 
فاصالً الجزء األفريقي منها وهو األكبر مساحة عن الجزء األسيوي األصغر 
منها"، تشكل الصحراء غالبية مساحة مصر، ويتركز أغلب سكان الجمهورية  
في وادى النيل والدلتا2. 
 كماجاء في دستورها الجديد( 2014 )،جمهورية مصر العربية دولة ذات سيادة، موحدة ال تقبل التجزئة، وال  
ينزل عن شيء منها، نظامها جمهوري  دي

# Step 5 - Embedding Model

In [12]:
model_id = "Alibaba-NLP/gte-multilingual-base"
dim = 768

device = "cuda:0"

In [15]:
pip install tf-keras --break-system-packages -q

Note: you may need to restart the kernel to use updated packages.


In [14]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(model_id, device=device)

embeddings = SentenceTransformerEmbeddings(
    model_name=model_id,
    model_kwargs={'device': device}
)

2025-09-11 15:08:05.850929: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

2025-09-11 15:08:08.288742: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

ValueError: Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.

In [ ]:
from sentence_transformers.util import batch_to_device
  
def batch_encode(texts, model, batch_size=16):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        emb = model.encode(batch, show_progress_bar=False)
        embeddings.extend(emb)
    return embeddings
 
encoded_docs = batch_encode(list(doc_texts.values()), model, batch_size=16)

In [ ]:
Data_docs = [doc for doc in documents]

Data_texts = []

for doc in Data_docs:
    Title = doc.metadata.get("title")
    Context = doc.metadata.get("context")

encoded_data = batch_encode(Data_texts, model, batch_size=16)

encoded_data = [e.tolist() for e in encoded_data]

# Step 6 - Create The Vector Database - ChromaDB

In [ ]:
chroma_client = chromadb.PersistentClient(path="./outputs/data/vector_database/vector_db")

In [ ]:
collection = chroma_client.create_collection(
    name="ar_docs",
    metadata={"hnsw:space": "cosine"}
)

In [ ]:
collection.add(
    documents=doc_text,
    embeddings=embedding_questions,
    metadatas=QA_metadata,
    ids=ids_to_add
)

In [ ]:
collection = chroma_client.get_or_create_collection(name="ar_docs")

In [ ]:
vectordb = Chroma(
    collection_name="ar_docs",
    embedding_function=embeddings,
    persist_directory="./outputs/vector_database/vector_db"
)

In [ ]:
question = "ما هي مظاهر الحياة السياسية بالدولة المصرية ؟"


question_embed = model.encode([question])[0].tolist()

results = collection.query(
    query_embeddings=[question_embed],
    n_results=3
)

print(results)

In [ ]:
if results["documents"]:
    top_match_metadata = results["metadatas"][0][0]
    Matched_Title = top_match_metadata.get("Title", "")
    Context = top_match_metadata.get("Context")

    print("Nearest Matched Titles: ", Matched_Title)
    print("Context Of The Nearest Titles:", Context)
    
else:
    print("No Find Any !!")

# Step 7 - Evaluation The embedding model

### Similarity search

In [ ]:
chroma_results = []
qa_embeddings = []  

for embedding in encoded_questions:
    results = vectordb.similarity_search_by_vector(
        embedding=embedding, 
        k=3
    )
    chroma_results.append(results)
    qa_embeddings.append(embedding)

### Accuracy

In [ ]:
chroma_insights = {
    "valid": 0,
    "similar": 0,
    "invalid": 0
}

for i in range(len(qa_texts)):
    true_metadata = QA_metadata[i]  

    pred_metadata = chroma_results[i][0].metadata

    true_id = str(true_metadata.get("id", ""))
    pred_id = str(pred_metadata.get("id", ""))

    true_source = true_metadata.get("source", "")
    pred_source = pred_metadata.get("source", "")

    # تصنيف النتائج
    if true_id == pred_id:
        chroma_insights["valid"] += 1
    elif true_source == pred_source:
        chroma_insights["similar"] += 1
    else:
        chroma_insights["invalid"] += 1

# حساب النسب
total = len(qa_texts)
chroma_insights["valid_percentage"] = chroma_insights["valid"] / total
chroma_insights["similar_percentage"] = chroma_insights["similar"] / total
chroma_insights["invalid_percentage"] = chroma_insights["invalid"] / total

# طباعة النتائج
print("Model ID:", model_id)
print("----")
print("Valid:", chroma_insights["valid"])
print("Valid%:", chroma_insights["valid_percentage"])
print("----")
print("Similar:", chroma_insights["similar"])
print("Similar%:", chroma_insights["similar_percentage"])
print("----")
print("Invalid:", chroma_insights["invalid"])
print("Invalid%:", chroma_insights["invalid_percentage"])
print("----")